In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('ggplot')
pd.set_option('display.width',2000)
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/milk-cheese-and-eggs-prices/Milk cheese and eggs prices.csv')
data

In [ ]:
data.groupby('Countries').size()

In [ ]:
data.info()

In [ ]:
data.duplicated().sum()

In [ ]:
data.isna().sum()

In [ ]:
data.describe()

In [ ]:
data.max()

In [ ]:
data.groupby('Milk, cheese, and eggs prices, 2017').max()

In [ ]:
data['Milk, cheese, and eggs prices, 2017'].mean()

In [ ]:
# creating the same in histplot as distplot is deprecated
plt.figure(figsize=(10,80))
for i in range(len(data.columns)):
    plt.subplot(17,1,i+1)
    sns.histplot(data[data.columns[i]],kde = True, stat = 'density',kde_kws = dict(cut = 3),edgecolor = None,color = 'g',alpha = 0.4, line_kws= { 'lw':3,'color':'blue'})
    plt.title(data.columns[i])
    
plt.tight_layout
plt.show()

In [ ]:
data['Countries'].unique()

In [ ]:
data['Countries'].value_counts()

In [ ]:
plt.figure(figsize = (20,10))
sns.boxplot(data)
plt.show()

In [ ]:
sns.pairplot(data)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
for columns in data.columns:
    if data[columns].dtypes == object:
        data[columns] = le.fit_transform(data[columns])
data
        

In [ ]:
data.corr()

In [ ]:
x = data.drop(["Available data"],axis = 1)
x

In [ ]:
plt.figure(figsize = (8,5))
sns.heatmap(x.corr(),annot = True)

In [ ]:
x.hist(bins = 50,figsize = (15,8))
plt.show()

In [ ]:
sc = StandardScaler()
sc = sc.fit_transform(x)
sc


In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 5)
km.fit(sc)
km.cluster_centers_
print(km.labels_)


In [ ]:

cluster = pd.concat([data,pd.DataFrame(km.labels_)],axis = 1)
cluster

In [ ]:
print(type(km.labels_))
print(len(km.labels_))

In [ ]:
unique,counts = np.unique(km.labels_,return_counts= True)
print(dict(zip(unique,counts)))

In [ ]:
data['cluster'] = km.labels_

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i,init = 'k-means++')
    kmeans.fit(sc)
    wcss.append(kmeans.inertia_)
wcss

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(range(1,11),wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('wcss')

In [ ]:
kmeans = KMeans(n_clusters = 5,init = 'k-means++')
kmeans = kmeans.fit(sc)
y_kmeans = kmeans.predict(sc)
y_kmeans

In [ ]:
y_kmeans.shape

In [ ]:
sc.shape

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_kmeans == 0, 0], sc[y_kmeans == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_kmeans == 1, 0], sc[y_kmeans == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_kmeans == 2, 0], sc[y_kmeans == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_kmeans == 3, 0], sc[y_kmeans == 3, 1], s=100, c='blue', label="Cluster 4")

# Scatterplot for Cluster 5
plt.scatter(sc[y_kmeans == 4, 0], sc[y_kmeans == 4, 1], s=100, c='cyan', label="Cluster 5")
plt.legend()
plt.show()


In [ ]:
sc

In [ ]:
import scipy.cluster.hierarchy as sch
plt.rcParams['figure.figsize'] = (20,10)
dendrogram = sch.dendrogram(sch.linkage(sc,method = 'centroid'))
plt.title("Dendrogram")
plt.xlabel('Milk, cheese, and eggs prices, 2017')
plt.ylabel('Global rank')

In [ ]:
import scipy.cluster.hierarchy as sch
plt.rcParams['figure.figsize'] = (20,10)
dendrogram = sch.dendrogram(sch.linkage(sc,method = 'single'))
plt.title("Dendrogram")
plt.xlabel('Milk, cheese, and eggs prices, 2017')
plt.ylabel('Global rank')

In [ ]:
import scipy.cluster.hierarchy as sch
plt.rcParams['figure.figsize'] = (20,10)
dendrogram = sch.dendrogram(sch.linkage(sc,method = 'ward'))
plt.title("Dendrogram")
plt.xlabel('Milk, cheese, and eggs prices, 2017')
plt.ylabel('Global rank')

In [ ]:
import scipy.cluster.hierarchy as sch
plt.rcParams['figure.figsize'] = (20,10)
dendrogram = sch.dendrogram(sch.linkage(sc,method = 'complete'))
plt.title("Dendrogram")
plt.xlabel('Milk, cheese, and eggs prices, 2017')
plt.ylabel('Global rank')

In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 5,affinity = 'euclidean',linkage = 'single')
y_ac = ac.fit_predict(sc)
y_ac

In [ ]:
pd.DataFrame(y_ac).value_counts()

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_ac == 0, 0], sc[y_ac == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_ac == 1, 0], sc[y_ac == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_ac == 2, 0], sc[y_ac == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_ac == 3, 0], sc[y_ac == 3, 1], s=100, c='blue', label="Cluster 4")

# Scatterplot for Cluster 5
plt.scatter(sc[y_ac == 4, 0], sc[y_ac == 4, 1], s=100, c='cyan', label="Cluster 5")
plt.legend()
plt.show()



In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 5,affinity = 'euclidean',linkage = 'ward')
y_ac = ac.fit_predict(sc)
y_ac

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_ac == 0, 0], sc[y_ac == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_ac == 1, 0], sc[y_ac == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_ac == 2, 0], sc[y_ac == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_ac == 3, 0], sc[y_ac == 3, 1], s=100, c='blue', label="Cluster 4")

# Scatterplot for Cluster 5
plt.scatter(sc[y_ac == 4, 0], sc[y_ac == 4, 1], s=100, c='cyan', label="Cluster 5")
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 5,affinity = 'euclidean',linkage = 'average')
y_ac = ac.fit_predict(sc)
y_ac

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_ac == 0, 0], sc[y_ac == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_ac == 1, 0], sc[y_ac == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_ac == 2, 0], sc[y_ac == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_ac == 3, 0], sc[y_ac == 3, 1], s=100, c='blue', label="Cluster 4")

# Scatterplot for Cluster 5
plt.scatter(sc[y_ac == 4, 0], sc[y_ac == 4, 1], s=100, c='cyan', label="Cluster 5")
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 5,affinity = 'euclidean',linkage = 'complete')
y_ac = ac.fit_predict(sc)
y_ac

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_ac == 0, 0], sc[y_ac == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_ac == 1, 0], sc[y_ac == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_ac == 2, 0], sc[y_ac == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_ac == 3, 0], sc[y_ac == 3, 1], s=100, c='blue', label="Cluster 4")

# Scatterplot for Cluster 5
plt.scatter(sc[y_ac == 4, 0], sc[y_ac == 4, 1], s=100, c='cyan', label="Cluster 5")
plt.legend()
plt.show()



In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 6,affinity = 'euclidean',linkage = 'complete')
y_ac = ac.fit_predict(sc)
y_ac

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_ac == 0, 0], sc[y_ac == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_ac == 1, 0], sc[y_ac == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_ac == 2, 0], sc[y_ac == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_ac == 3, 0], sc[y_ac == 3, 1], s=100, c='blue', label="Cluster 4")

# Scatterplot for Cluster 5
plt.scatter(sc[y_ac == 4, 0], sc[y_ac == 4, 1], s=100, c='cyan', label="Cluster 5")

# Scatterplot for Cluster 6
plt.scatter(sc[y_ac == 5, 0], sc[y_ac == 5, 1], s=100, c='orange', label="Cluster 6")
plt.legend()

plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 4,affinity = 'euclidean',linkage = 'complete')
y_ac = ac.fit_predict(sc)
y_ac

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_ac == 0, 0], sc[y_ac == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_ac == 1, 0], sc[y_ac == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_ac == 2, 0], sc[y_ac == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_ac == 3, 0], sc[y_ac == 3, 1], s=100, c='blue', label="Cluster 4")


plt.legend()

plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters = 8,affinity = 'euclidean',linkage = 'complete')
y_ac = ac.fit_predict(sc)
y_ac

In [ ]:
plt.figure(figsize=(10, 8))



# Scatterplot for Cluster 1
plt.scatter(sc[y_ac == 0, 0], sc[y_ac == 0, 1], s=100, c='red', label="Cluster 1")

# Scatterplot for Cluster 2
plt.scatter(sc[y_ac == 1, 0], sc[y_ac == 1, 1], s=100, c='magenta', label="Cluster 2")

# Scatterplot for Cluster 3
plt.scatter(sc[y_ac == 2, 0], sc[y_ac == 2, 1], s=100, c='green', label="Cluster 3")

# Scatterplot for Cluster 4
plt.scatter(sc[y_ac == 3, 0], sc[y_ac == 3, 1], s=100, c='blue', label="Cluster 4")

# Scatterplot for Cluster 5
plt.scatter(sc[y_ac == 4, 0], sc[y_ac == 4, 1], s=100, c='cyan', label="Cluster 5")

# Scatterplot for Cluster 6
plt.scatter(sc[y_ac == 5, 0], sc[y_ac == 5, 1], s=100, c='orange', label="Cluster 6")

# Scatterplot for Cluster 7
plt.scatter(sc[y_ac == 6, 0], sc[y_ac == 6, 1], s=100, c='brown', label="Cluster 7")

# Scatterplot for Cluster 8
plt.scatter(sc[y_ac == 7, 0], sc[y_ac == 7, 1], s=100, c='purple', label="Cluster 8")
plt.legend()

plt.show()
